# DL 심화 과제

2020.01.30 18기 문헌정보학과 김채형

In [1]:
# import libs
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
# reproducibility
torch.manual_seed(99)

In [3]:
# load datasets
mnist_train = dsets.MNIST(root='MNIST_data/', 
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', 
                         train=False, 
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
# parameters
batch_size = 100
training_epochs = 10
learning_rate = 0.001

In [5]:
# data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          drop_last=True)

In [6]:
# model
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)   
        return out

In [7]:
model = CNN()

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished!')        

[Epoch: 1] cost = 0.1625555455684662
[Epoch: 2] cost = 0.04230440780520439
[Epoch: 3] cost = 0.031035028398036957
[Epoch: 4] cost = 0.021753555163741112
[Epoch: 5] cost = 0.017752543091773987
[Epoch: 6] cost = 0.014042164199054241
[Epoch: 7] cost = 0.012021452188491821
[Epoch: 8] cost = 0.011838998645544052
[Epoch: 9] cost = 0.01008572243154049
[Epoch: 10] cost = 0.00743548758327961
Learning Finished!


In [10]:
# test
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float()
    Y_test = mnist_test.targets
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: {}'.format(accuracy.item()))

Accuracy: 0.9866999983787537
